# Sample dataset

In [36]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [37]:
diabetes = load_diabetes()

In [38]:
X = pd.DataFrame(diabetes.data,columns=diabetes.feature_names)
X

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641
...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930


In [39]:
y = pd.Series(diabetes.target,name='Disease_Progression')
y

0      151.0
1       75.0
2      141.0
3      206.0
4      135.0
       ...  
437    178.0
438    104.0
439    132.0
440    220.0
441     57.0
Name: Disease_Progression, Length: 442, dtype: float64

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [41]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 1. Using math formula

In [42]:
lambda_param = 0.1    # Hệ số điều chuẩn (λ)
# Tính các hệ số Ridge Regression
I = np.eye(X_train.shape[1])  # Ma trận đơn vị (kích thước: số cột của X_bias)
I[0,0] = 0  # Không điều chuẩn thành phần bias
# Công thức Ridge Regression
coefficients = np.dot(
    np.linalg.inv(np.dot(X_train.T,X_train)+lambda_param*I),  # Ma trận nghịch đảo
    np.dot(X_train.T, y_train)                                        # Nhân ma trận
)
coefficients

array([  1.76114425, -11.50497111,  25.62468688,  16.81767375,
       -43.20958592,  23.67750275,   7.13078061,  12.96539345,
        34.68244486,   2.36387901])

In [43]:
# 1. Thêm cột toàn số 1 vào trước ma trận X để tạo X*
X_star = np.hstack([np.ones((X_train.shape[0], 1)), X_train])

Q = np.eye(X_star.shape[1])
Q[0,0] = 0

# 2. Tính hệ số w bằng công thức bình phương tối thiểu
w = np.dot(
    np.linalg.inv(np.dot(X_star.T, X_star)+lambda_param*Q),
    np.dot(X_star.T, y_train)
)
w[0]

153.73654390934846

In [44]:
a = np.dot(X_test,coefficients)+w[0]
a

array([139.58542157, 179.57697526, 134.250153  , 291.51284708,
       123.71047445,  92.22657565, 258.18865821, 181.54369082,
        90.00924495, 108.71919814,  94.17405616, 168.23084537,
        53.86212503, 206.58637287, 100.08765697, 130.72540768,
       219.60842407, 250.64149071, 196.35580077, 218.43951614,
       207.34390244,  88.49981985,  70.50145835, 188.9364125 ,
       154.93633886, 159.45690156, 188.36170612, 180.27242173,
        48.06216612, 109.03265377, 174.97462553,  86.54422395,
       132.90698012, 184.40484987, 173.81004816, 190.38616734,
       124.34663813, 119.5926622 , 147.86809238,  59.12772405,
        71.73500769, 107.69477845, 165.33120469, 154.81882564,
       171.19537515,  61.60733761,  71.90717574, 114.70091966,
        51.82371763, 167.36048838, 152.67883591,  63.0467208 ,
       103.86147935, 109.17307803, 175.44722226, 154.79223522,
        94.42297062, 210.6391186 , 120.18235089,  77.32234516,
       187.82474798, 206.35676525, 140.66952475, 105.58

In [45]:
b = np.mean((y_test-a)**2)
b

2899.066814017104

## 2. Using created model

In [46]:
class RidgeRegression:
    def __init__(self,learning_rate=0.01,n_iterations=1000,alpha=0.1):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.alpha = alpha        # Regularization strength
        self.weights = None
        self.bias = None
    def fit(self,X,y):
        n_samples,n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        for _ in range(self.n_iterations):
            # Predict
            y_predicted = np.dot(X,self.weights)+self.bias
            # Calculate gradient (include regularization)
            dw = (1/n_samples)*np.dot(X.T,(y_predicted-y))+(self.alpha*self.weights)/n_samples
            db = (1/n_samples)*np.sum(y_predicted-y)
            # Update parameters
            self.weights = self.weights-self.learning_rate*dw
            self.bias = self.bias-self.learning_rate*db
    def predict(self,X):
        return np.dot(X,self.weights)+self.bias
    def r2_score(self,y_true,y_pred):
        """
        Tính toán R² score để đánh giá mô hình
        """
        ss_total = np.sum((y_true-np.mean(y_true))**2)
        ss_residual = np.sum((y_true-y_pred)**2)
        return 1-(ss_residual/ss_total)
    def mean_square_error(self,y_true,y_pred):
        """
        Tính toán Mean Square Error (MSE)
        """
        mse = np.mean((y_true-y_pred)**2)
        return mse

In [47]:
model = RidgeRegression()
model.fit(X_train,y_train)

In [48]:
y_pred = model.predict(X_test)
y_pred

array([140.60433878, 181.71959059, 140.38132173, 294.51366947,
       120.98139183,  93.39020425, 257.3925688 , 187.76972127,
        83.27580018, 111.00084103,  95.02388167, 162.57250546,
        64.28088212, 205.38363737,  98.53143389, 132.39277603,
       222.05681109, 246.63365793, 196.46921981, 214.53290762,
       207.77966077,  88.64349652,  71.92873049, 188.47326077,
       156.69653387, 162.32934101, 190.0801412 , 176.79848564,
        49.75496324, 110.47187482, 180.90391768,  91.60363189,
       131.35232626, 180.48499126, 173.15098871, 191.57737699,
       122.13163997, 117.55038998, 145.03853976,  60.78775935,
        74.43689421, 107.61102949, 161.90813517, 148.66955251,
       175.73891338,  65.53343236,  78.79615632, 106.42146109,
        58.63744953, 161.2023525 , 157.68932385,  65.62731583,
       114.10680687, 107.89658099, 169.46391033, 160.70088349,
        94.18034418, 208.0051284 , 118.04392795,  68.03231106,
       184.67389835, 202.84161398, 141.78748224, 104.84

In [49]:
r2 = model.r2_score(y_test,y_pred)
r2

0.45550545514528795

In [50]:
mse = model.mean_square_error(y_test,y_pred)
mse

2884.8147248797436